In [75]:
import json
import csv
states = ["ak","al","ar","az","ca","co","ct","de","fl","ga","hi","ia","id","il",
          "in","ks","ky","la","ma","md","me","mi","mn","mo","ms","mt","nc","nd","ne","nh",
          "nj","nm","nv","ny","oh","ok","or","pa","ri","sc","sd","tn","tx","ut","va","vt",
          "wa","wi","wv","wy"]
states_names = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC':'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
'SC': 'South Carolina', 'SD': 'South Dakota', 'TN':'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
'WA': 'Washington','WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}


In [80]:
#Leer los tweets del fichero output.txt y guardarlos en la variable tweets
with open('output.txt') as json_file:
    tweets = json.load(json_file)

    
def getState(data):
    if data["place"] != None and data["place"]["country_code"] == "US":
        state = str(data["place"]["full_name"]).lower().split(", ")
        if len(state) > 1:
            return state[1]

In [77]:
#reading the tweet data, check for states and non-empty tweets, 
#add those to sent_dict, where a key is a state and the value
#now we analyze those tweets
from afinn import Afinn

afinn = Afinn()
sent_dict = {}

score_dict = dict.fromkeys(sent_dict.keys())
for entry in sent_dict:
    #scoring each set of tweets from each state
    total = 0
    for tweet in sent_dict[entry]:
        #calculating the normalized happiness for the tweets
        score = afinn.score(tweet)/len(sent_dict[entry])
        total = total + score
    score_dict[entry] = total
#add the states without tweets with a happiness value of 0
for state in states:
    if state not in score_dict.keys():
        score_dict[state] = 0
#and returning the 5 happiest states as well as the score_dict
result = sorted(score_dict, key=score_dict.get, reverse=True)[:5],score_dict

In [78]:
#saving the results and sorting them

for entry in tweets:
        if "created_at" in entry:
            state = getState(entry)
            if state in states:
                if "text" in entry:
                    if state not in sent_dict.keys():
                        sent_dict[state] = [entry["text"]]
                    else:
                        sent_dict[state].append(entry["text"])

result = evaluateDict()
print("happiest five states are "+str(result[0]))

#sorting the keys descending and readding the values so that
#the map can be populated properly
score_dict = result[1]
buffer = {}
for key in sorted(score_dict):
    buffer[key]=score_dict[key]
score_dict = buffer

happiest five states are ['ky', 'sc', 'vt', 'mo', 'wa']


In [79]:
#painting the USA graph
import plotly.graph_objects as go
import plotly
global score_dict

# Load data frame and tidy it.
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

#generating the plit
fig = go.Figure(data=go.Choropleth(
    locations=df['code'], # Spatial coordinates
    z = list(score_dict.values()),# our sentiment data
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Portland',
    colorbar_title = 'Grado de felicidad',
))

#adding layout
fig.update_layout(
    title_text = 'Estados más felices de EEUU',
    geo_scope='usa', # limite map scope to USA
)

fig.write_html("file.html")